# Imports


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd
from time import time
from sp_sims.simulators.stochasticprocesses import BDStates
from samprecon.environments.OneEpisodeEnvironments import MarkovianUniformEnvironment
from samprecon.samplers.agents import SimpleAgent 
from samprecon.reconstructors.NNReconstructors import RNNReconstructor
from tqdm.notebook import tqdm

plt.style.use('rose-pine-dawn')
rnd.seed(int(time()))
%load_ext autoreload
%autoreload 2

In [2]:
# Generate Environments on which to learn 
high_res_delta = 1e-1 # For generating the dataset and later sample
baseline_rates = {"lam": 1/10,"mu":4/10}
length_path = 64 # High-res path length
max_decimation = length_path//2 # Max decimation factor #CHECK: Maybe not divide by 2
starting_dec_rate = 1
epochs = 100
lenth_of_episode = 12
step_path_length = 1
sampling_budget = 4
used_path_length = 64 # So that we can let the process reach stationarity and take samples from stationary distribution
num_states = 4
current_decimation_factor = int(# We can start somewhere in between 
    np.mean(1/np.array(
            list(baseline_rates.values())
    ))//high_res_delta)
print(f"Decimation factor is {current_decimation_factor}")
# Set random seed with time for randomnessj

Decimation factor is 62


In [3]:
# Initialize context first
stateGen = BDStates(baseline_rates, high_res_delta, num_states)
sampling_arbiter = SimpleAgent(num_states,max_decimation)
reconstructor = RNNReconstructor(subsampled_signal_length = sampling_budget )

env = MarkovianUniformEnvironment(
    state_generator = stateGen,
    sampling_arbiter = sampling_arbiter,
    reconstructor = reconstructor, 
    starting_decrate = starting_dec_rate,
)

This is bothering you : 1


AttributeError: 'int' object has no attribute 'squeeze'

In [ ]:
ebar = tqdm(range(epochs), desc = "Epochs")
for epoch in range(epochs):
    # We generate a single step from the generator process
    sbar = tqdm(range(lenth_of_episode), desc = "Steps")
    for step in range(lenth_of_episode):

        loss = env.step() 
        sbar.set_description(f"Loss: {loss}")

        sbar.update(1)
    ebar.update(1)
    # We get reward based on how close we got to maximum information 

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Steps:   0%|          | 0/12 [00:00<?, ?it/s]

This is bothering you : tensor([[15.7829]], grad_fn=<ViewBackward0>)


/Users/rac716/Research/SamplingReconstruction/samprecon/samplers/spatial_transformers.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(decimation_intervals[i])


TypeError: only integer tensors of a single element can be converted to an index